## Ingestion and Updation within the vector db

In [1]:
import json
import chromadb
from openai import OpenAI

In [2]:
chroma_client = chromadb.PersistentClient(path="./chroma_db/")
collection = chroma_client.get_or_create_collection(name="it_tickets", metadata={"hnsw:space": "cosine"})
client = OpenAI()

def get_embeddings(text: str):
	response = client.embeddings.create(
		input=text,
		model="text-embedding-3-large"
	)
	return response.data[0].embedding

In [3]:
with open('sample_tickets.json', 'r') as f:
	tickets = json.load(f)
for ticket in tickets:
	ticket['ticket_info'] = f"{ticket['title']}: {ticket['description']}"
	ticket_id = ticket['id']
	embedding = get_embeddings(ticket['ticket_info'])
	contributors = ticket.get('contributors', [])
	metadata = {
		"title": ticket['title'],
		"description": ticket['description'],
		"status": ticket['status'],
		"contributors": json.dumps(contributors)
	}
	collection.add(
		ids=[ticket_id],
		documents=[ticket['ticket_info']],
		metadatas=[metadata],
		embeddings=[embedding]
	)
	
print("Ingestion complete.")

Ingestion complete.


In [4]:
def search_tickets(query: str, n_results: int = 3):
    """Search tickets using the same embedding model"""
    query_embedding = get_embeddings(query)
    
    # Search using the embedding
    results = collection.query(
        query_embeddings=[query_embedding],  
        n_results=n_results,
        include=["documents", "metadatas", "distances"]
    )
    
    return results

results = search_tickets("database connection timeout")
results

{'ids': [['tkt-105', 'tkt-100', 'tkt-109']],
 'embeddings': None,
 'documents': [['Database connection timeout: Database server taking too long to respond to queries.',
   'Network latency issue: Observed abnormal response times in the internal network.',
   'Slow response from API: API endpoint returning responses with delay > 3s.']],
 'uris': None,
 'included': ['documents', 'metadatas', 'distances'],
 'data': None,
 'metadatas': [[{'status': 'resolved',
    'description': 'Database server taking too long to respond to queries.',
    'contributors': '[{"contributor_name": "David", "action_taken": "Rebooted server", "timestamp": "2025-09-01T14:00:00Z"}]',
    'title': 'Database connection timeout'},
   {'title': 'Network latency issue',
    'status': 'resolved',
    'contributors': '[{"contributor_name": "Frank", "action_taken": "Ran diagnostics", "timestamp": "2025-09-01T09:00:00Z"}, {"contributor_name": "David", "action_taken": "Replaced hardware component", "timestamp": "2025-09-01

In [ ]:
print(collection.count())

10


### Updation

In [ ]:
new_ticket_id = "tkt-106"
new_ticket_desc = "Power outage in server room affecting multiple racks and causing network downtime."
new_ticket_title = "Server room power outage"
new_ticket_status = "open"
contributors = [
    {
        "contributor_name": "Grace",
        "action_taken": "Reported outage",
        "timestamp": "2025-09-01T20:00:00Z"
    }
]

new_ticket_embedding = get_embeddings(new_ticket_desc) 


new_ticket_metadata = {
    "title": new_ticket_title,
    "status": new_ticket_status,
    "contributors": json.dumps(contributors)
}

collection.upsert(
    ids=[new_ticket_id],
    documents=[new_ticket_desc],
    metadatas=[new_ticket_metadata],
    embeddings=[new_ticket_embedding]
)

print(f"Ticket {new_ticket_id} upserted with manual embedding.")
print("New total count:", collection.count())

Ticket tkt-106 upserted with manual embedding.
New total count: 10


In [ ]:
all_items = collection.get(
    include=["documents", "metadatas"]
)

print(f"Total items: {len(all_items['ids'])}")
for i, (doc_id, doc, meta) in enumerate(zip(all_items['ids'], all_items['documents'], all_items['metadatas'])):
    print(f"\n{i+1}. ID: {doc_id}")
    print(f"   Document: {doc}")
    print(f"   Metadata: {meta}")

Total items: 10

1. ID: tkt-100
   Document: Network latency issue: Observed abnormal response times in the internal network.
   Metadata: {'contributors': '[{"contributor_name": "Frank", "action_taken": "Ran diagnostics", "timestamp": "2025-09-01T09:00:00Z"}, {"contributor_name": "David", "action_taken": "Replaced hardware component", "timestamp": "2025-09-01T09:15:00Z"}]', 'title': 'Network latency issue', 'status': 'resolved'}

2. ID: tkt-101
   Document: Disk space running low: Server /var partition is over 90% full.
   Metadata: {'contributors': '[{"contributor_name": "Bob", "action_taken": "Reconfigured settings", "timestamp": "2025-09-01T10:00:00Z"}, {"contributor_name": "Eve", "action_taken": "Tested connectivity", "timestamp": "2025-09-01T10:15:00Z"}]', 'title': 'Disk space running low', 'status': 'resolved'}

3. ID: tkt-102
   Document: CPU overheating: CPU temperature has exceeded safe operating limits.
   Metadata: {'status': 'resolved', 'title': 'CPU overheating', 'contrib

## Problem resolution with Agent

In [ ]:
from langchain_groq import ChatGroq
from langchain.agents import AgentType, initialize_agent
from langchain.agents import Tool
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.utilities import SearchApiAPIWrapper
# from langchain.memory import ConversationBufferWindowMemory
# from langchain.prompts import MessagesPlaceholder
# from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
import json

# Initialize LLM
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

def ticket_lookup(query: str):
    """
    Look up IT support tickets from the database.
    Returns formatted ticket information or clear message if none found.
    """
    try:
        results = collection.query(
            query_texts=[query],
            n_results=2,
            include=["documents", "metadatas"]
        )
        
        output = ""
        documents = results.get("documents")
        metadatas = results.get("metadatas")
        ids = results.get("ids")
        
        if documents and metadatas and len(documents[0]) > 0:
            for id, doc, meta in zip(ids[0], documents[0], metadatas[0]):
                contributors_str = meta.get("contributors", "[]")
                contributors = []
                try:
                    for c in json.loads(contributors_str):
                        name = c.get("contributor_name", "")
                        action = c.get("action_taken", "")
                        contributors.append(f"{name}: {action}")
                except json.JSONDecodeError:
                    contributors = ["Unable to parse contributors"]
                
                output += (f"TICKET ID: {id}\n"
                          f"Title: {meta.get('title', 'No title')}\n"
                          f"Description: {doc}\n"
                          f"Contributors: {contributors}\n"
                          f"Status: {meta.get('status', 'Unknown')}\n"
                          f"{'='*40}\n")
        
        if output:
            return f"FOUND TICKETS:\n{output}"
        else:
            return "NO_TICKETS_FOUND: No relevant tickets found in the database for this query."
    
    except Exception as e:
        return f"ERROR: Unable to search ticket database - {str(e)}"

def web_search_wrapper(query: str):
    """
    Wrapper for web search with clear output formatting
    """
    try:
        search = SearchApiAPIWrapper()
        results = search.run(query)
        return f"WEB_SEARCH_RESULTS:\n{results}"
    except Exception as e:
        return f"ERROR: Web search failed - {str(e)}"

# Initialize vector store and retriever
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="./chroma_data/", embedding_function=embedding_function)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Define tools with very specific descriptions
tools = [
    Tool(
        name="ticket_database",
        func=ticket_lookup,
        description="""Search for similar past IT support tickets. Use this FIRST for any IT issue.
        Input: A descriptive query about the IT problem (e.g., 'database slow queries', 'email sync issues').
        Output: Returns ticket details or 'NO_TICKETS_FOUND' message."""
    ),
    Tool(
        name="web_search", 
        func=web_search_wrapper,
        description="""Search the web for current IT troubleshooting information. 
        Use ONLY if ticket_database returned 'NO_TICKETS_FOUND' or you need additional current information.
        Input: Specific technical search terms."""
    )
]

# Create a proper system prompt
system_prompt = """You are an IT support assistant. Follow this EXACT process:

1. FIRST: Always use ticket_database to search for similar past tickets
2. IF tickets found: Analyze them and provide a complete response with:
   - Issue summary
   - Prioritized troubleshooting steps (synthesized from tickets, not copied)
   - Escalation guidance if critical
3. IF no tickets found (NO_TICKETS_FOUND): Then use web_search for additional information
4. Provide ONE comprehensive response - do NOT search multiple times for the same issue

IMPORTANT: After using tools, provide your final answer. Do NOT keep searching repeatedly."""

# Initialize agent with loop prevention
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,  # Limit iterations to prevent loops
    early_stopping_method="generate",
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix": system_prompt,
        "format_instructions": """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [ticket_database, web_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat at most 2 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""
    }
)

# Example usage function with additional safety
def handle_user_query(user_input: str, max_retries=1):
    """
    Process user query with retry logic and loop detection
    """
    for attempt in range(max_retries + 1):
        try:
            print(f"Processing query (attempt {attempt + 1}): {user_input}")
            response = agent.run(user_input)
            
            # Check if response seems complete
            if len(response.strip()) > 20:  # Basic completeness check
                return response
            else:
                print("Response seems incomplete, retrying...")
                continue
                
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries:
                return f"I encountered an error processing your request: {str(e)}. Please try rephrasing your question or contact IT support directly."
    
    return "Unable to process request after multiple attempts."

# Test the system
if __name__ == "__main__":
    query = "New issue: Database server taking too long to respond to queries."
    response = handle_user_query(query)
    print("\n" + "="*50)
    print("FINAL AGENT RESPONSE:")
    print("="*50)
    print(response)

Processing query (attempt 1): New issue: Database server taking too long to respond to queries.


> Entering new AgentExecutor chain...
Action: ticket_database
Action Input: 'database slow queries'
Observation: FOUND TICKETS:
TICKET ID: tkt-203
Title: Slow query performance
Description: Database queries taking longer than usual in production.
Contributors: []
Status: open
TICKET ID: tkt-108
Title: High memory usage on server
Description: Unusually high RAM consumption noted on production server.
Contributors: ['Bob: Applied patch', 'Eve: Reconfigured settings', 'Frank: Replaced hardware component']
Status: open

Thought:Action: Analyze the found tickets
Action Input: Analyze the details of tkt-203 and tkt-108
Observation: Analyze the found tickets is not a valid tool, try one of [ticket_database, web_search].
Thought:Action: Analyze the found tickets
Action Input: Synthesize the details of tkt-203 and tkt-108 to provide a complete response
Observation: Analyze the found tickets is not 

## Problem resolution without Agent

In [ ]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.PersistentClient(path="./chroma_data/")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
collection = client.get_or_create_collection(name="it_tickets")

In [ ]:
# results = collection.query(
# 	query_texts=["Database server taking too long to respond to queries."],
#   	n_results=2,
# 	include=["documents", "metadatas"]
# )

# results       

In [ ]:
# for id, description, meta in zip(results['ids'][0], results['documents'][0], results['metadatas'][0]):
# 	print("ID:", id)
# 	print("Description:", description)	
# 	# print("Metadata:", meta)
# 	print("Title:", meta.get("title"))
# 	print("Contributors:", json.loads(meta.get("contributors", "[]")))
# 	print("Status:", meta.get("status"))
# 	print("---")

In [ ]:
def ticket_lookup(query: str):				
    results = collection.query(
        query_texts=[query],
		where={"status": "open"},
		include=["documents", "metadatas"]
    )
    output = ""
    documents = results.get("documents")
    metadatas = results.get("metadatas")
    ids = results.get("ids")
    if documents and metadatas:
        for id, doc, meta in zip(ids[0], documents[0], metadatas[0]):
            contributors_str = meta.get("contributors", "[]")
            contributors = []
            for c in json.loads(contributors_str):
                name = c.get("contributor_name", "")
                action = c.get("action_taken", "")
                contributors.append(f"{name}: {action}")
            output += (f"ID: {id}\n"
                       f"Title: {meta.get('title')}\n"
					   f"Description: {doc}\n"
					   f"Contributors: {contributors}\n"
					   f"Status: {meta.get('status')}\n"
					   "----\n")
    return output if output else "No relevant tickets found."

In [41]:
print(ticket_lookup("Database server taking too long to respond to queries."))

ID: tkt-203
Title: Slow query performance
Description: Database queries taking longer than usual in production.
Contributors: []
Status: open
----
ID: tkt-109
Title: Slow response from API
Description: API endpoint returning responses with delay > 3s.
Contributors: ['Eve: Cleared cache', 'Eve: Escalated issue', 'Charlie: Checked logs', 'Frank: Checked logs']
Status: open
----



In [ ]:
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

load_dotenv()
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0)

template = """
You are an IT support assistant. 
You have the following similar tickets:

{tickets}

Your job:
- Summarize the likely issue in one paragraph.
- Suggest clear, actionable troubleshooting steps.
- Prioritize the steps.
- Include escalation guidance if needed.
- Write in plain language, do not repeat contributor actions verbatim.

Output a complete troubleshooting guide.
"""
prompt = PromptTemplate(
    input_variables=["tickets"],
    template=template
)

tickets_text = ticket_lookup("Database server taking too long to respond to queries.")  # raw ticket info
response = llm.invoke([
    {"role": "user", "content": prompt.format(tickets=tickets_text)}
])
print(response.content)

**Troubleshooting Guide – Slow Query / API Response**

**1. Likely Root Cause**  
The symptoms point to a performance bottleneck in the data layer. Either database queries are taking too long (missing indexes, complex joins, or high load) or the API layer is adding extra latency (caching, network hops, or resource contention). The two tickets you referenced both involve delays that exceed normal thresholds, so the first step is to confirm where the slowdown originates.

---

### Step‑by‑Step Troubleshooting (Prioritized)

| # | Action | Why It Matters | How to Do It |
|---|--------|----------------|--------------|
| **1** | **Check API response times in real‑time** | Confirms that the API is indeed slow and isolates the problem to the API layer. | Use a monitoring tool (e.g., New Relic, Datadog, or a simple `curl -w` script) to capture response times for the affected endpoint. |
| **2** | **Inspect API logs for errors or timeouts** | Errors or repeated retries can inflate response time